In [ ]:
def experiment_tcr(pattern = ".*", to_normalize = True, dump = False):
    
    tcr_1 = make_scorer(total_cost_ratio, price = 1, greater_is_better=True)
    tcr_9 = make_scorer(total_cost_ratio, price = 9, greater_is_better=True)
    tcr_99 = make_scorer(total_cost_ratio, price = 99, greater_is_better=True)

    t0 = time.time()
    pipes, recall_avg, recall_std, precision_avg, precision_std, tcr_1_avg, tcr_9_avg, tcr_99_avg = init()

    for estimator in pipes:
        name = estimator[0]
        if re.match(pattern, name) is not None:
            print("Trying: " + name + " ...")
            for cost in costs:
                extractor = estimator[1].steps[0][1].fit(X)
                if "Naive Bayes" in name:
                    model = estimator[1].set_params(clf__class_prior=[1/cost, (cost-1)/cost]).steps[1][1]
                elif "Random Forest" in name:
                    model = estimator[1].set_params(clf__class_weight={1:cost-1}).steps[1][1]
                elif "SVM" in name:
                    model = estimator[1].set_params(clf__class_weight={1:cost-1}).steps[1][1]
                elif "Baseline" in name:
                    model = estimator[1].steps[1][1]
            
                tcrs_1 = []
                tcrs_9 = []
                tcrs_99 = []
                for train_index, test_index in sss:
                    X_train, X_test = \
                        extractor.transform(X)[train_index], extractor.transform(X)[test_index]
                    y_train, y_test = y[train_index], y[test_index]
                    model.fit(X_train, y_train)
                    tcrs_1.append(tcr_1(model, X_test, y_test))
                    tcrs_9.append(tcr_9(model, X_test, y_test))
                    tcrs_99.append(tcr_99(model, X_test, y_test))
                tcr_1_avg[name].append(np.average(tcrs_1))
                tcr_9_avg[name].append(np.average(tcrs_9))
                tcr_99_avg[name].append(np.average(tcrs_99))
            if dump == True:
                write_results(name, precision_avg[name], recall_avg[name])
                
    t1 = time.time()
    print()
    print("Execution time: %.3f min" % ((t1 - t0)/60))
    
    
    return tcr_1_avg, tcr_9_avg, tcr_99_avg

In [ ]:
tcr_1_avg, tcr_9_avg, tcr_99_avg = experiment_tcr(dump = False)